## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token,  # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


In [2]:
model_name = "Waktaverse-Llama-3-KO-8B-Instruct"  # ADD YOUR MODEL NAME HERE
username = "PathFinderKR"  # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_name}"  # repository id

## Login to Weights & Biases

In [3]:
import wandb

api_key = os.getenv("WANDB_API_KEY")
wandb.login(
    key=api_key  # ADD YOUR API KEY HERE
)
wandb.init(project=model_name)

wandb: Currently logged in as: pathfinderkr. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/pathfinder/.netrc


## Imports

In [4]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# datasets
from datasets import load_dataset

## Device

In [5]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [6]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [7]:
################################################################################
# seed
################################################################################
seed=42
torch.manual_seed(seed)

################################################################################
# Tokenizer parameters
################################################################################
max_length=8192
padding="do_not_pad" # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True # True for sampling, False for greedy decoding
temperature=0.6
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4" # "nf4", #fp4"
bnb_4bit_use_double_quant=False

################################################################################
# LoRA parameters
################################################################################
task_type="CAUSAL_LM"
target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
r=16
lora_alpha=32
lora_dropout=0.1
bias="none"

################################################################################
# Training parameters
################################################################################
output_dir="./results"
logging_dir="./logs"
save_strategy="epoch"
logging_strategy="steps" # "steps", "epoch"
if logging_strategy == "steps":
    logging_steps=10
else:
    logging_steps=None
save_total_limit=1
report_to="wandb"

num_train_epochs=2
per_device_train_batch_size=4
gradient_accumulation_steps=2
gradient_checkpointing=True
learning_rate=2e-5
lr_scheduler_type="cosine" # "constant", "linear", "cosine"
warmup_ratio=0.1
optim = "adamw_torch"
weight_decay=0.1

################################################################################
# SFT parameters
################################################################################
max_seq_length=4096
packing=True # packing not supported for masking instructions

## Tokenizer

In [8]:
# Korean Tokenizer ID
korean_tokenizer_id = "saltlux/Ko-Llama3-Luxia-8B"

In [9]:
# Load tokenizer
korean_tokenizer = AutoTokenizer.from_pretrained(korean_tokenizer_id)

In [10]:
# Vocabulary size
print(f"Korean Tokenizer vocabulary size: {len(korean_tokenizer)}")

Korean Tokenizer vocabulary size: 145792


In [11]:
# Special tokens
print(korean_tokenizer.special_tokens_map)

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>', 'additional_special_tokens': ['<|none1|>']}


In [12]:
# Padding side
print(korean_tokenizer.padding_side)

right


## Model

In [13]:
# Model ID for base model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [14]:
# Quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [15]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
# Resize model embeddings
model.resize_token_embeddings(len(korean_tokenizer))

Embedding(145792, 4096)

In [17]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(145792, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=145792, bias=False)
)```

In [18]:
# Number of parameters
print(f"Number of parameters (in billions): {model.num_parameters() / 1e9:.2f}")

Number of parameters (in billions): 8.17


In [19]:
# Prepare model for kbit training
model = prepare_model_for_kbit_training(model)

## Dataset

In [20]:
# Dataset ID
dataset_id = "MarkrAI/KoCommercial-Dataset"

In [21]:
# Load the dataset
dataset = load_dataset(dataset_id)

In [22]:
# Dataset information
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 175454
    })
})

In [23]:
# Dataset example
print(dataset["train"][0]["instruction"])
print(dataset["train"][0]["input"])
print(dataset["train"][0]["output"])

보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.

모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \frac{1}{3}$, $P(B) = \frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \frac{1}{3} - frac{5}{12} = \frac{12}{12} - frac{4}{12} - frac{5}{12} = \frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \frac{1}{4}$입니다.


## Preprocessing

In [24]:
# Train on only a subset of the dataset for demonstration purposes
#dataset["train"] = dataset["train"].shuffle(seed=seed).select(range(100))

In [25]:
# Shuffle the dataset
dataset = dataset.shuffle(seed=seed)

In [26]:
# Alpaca dataset format: 
# {"instruction": [str],
#   "input": [str],
#   "output": [str]}

# Korean
def prompt_without_input(example):
    text = (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        "다음은 작업을 설명하는 지시사항입니다. 요청을 적절하게 완료하는 응답을 작성하세요.<|eot_id|>"
        
        "<|start_header_id|>user<|end_header_id|>\n\n"
        f"{example['instruction']}<|eot_id|>"
        
        "<|start_header_id|>assistant<|end_header_id|>\n\n"
        f"{example['output']}<|eot_id|>"
        )
    return {'text': text}
    
def prompt_with_input(example):
    text = ( 
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        "다음은 작업을 설명하는 지시사항과, 함께 쌍을 이루어 제공되는 입력입니다. 요청을 적절하게 완료하는 응답을 작성하세요.<|eot_id|>"
        
        "<|start_header_id|>user<|end_header_id|>\n\n"
        f"{example['instruction']}"
        f"{example['input']}<|eot_id|>"
        
        "<|start_header_id|>assistant<|end_header_id|>\n\n"
        f"{example['output']}<|eot_id|>"
        )
    return {'text': text}

In [27]:
def create_alpaca_prompt(example):
    # if input is not provided
    if example["input"] == "":
        return prompt_without_input(example)
    # if input is provided
    else:
        return prompt_with_input(example)

In [28]:
# Apply the alpaca prompt to the dataset
dataset = dataset.map(create_alpaca_prompt)

In [29]:
# Display the first example
print(dataset["train"][0]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음은 작업을 설명하는 지시사항입니다. 요청을 적절하게 완료하는 응답을 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

정진영은 어떤 분야에서 활동을 했나요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

정진영은 1964년 11월 19일에 태어난 대한민국의 배우로, 1988년 뮤지컬 배우로 데뷔했고 1989년 연극 배우로 데뷔했다. 그는 30년 동안 깊이 있는 연기력으로 관객들의 사랑을 받았다. 그의 대표적인 작품으로는 '왕의 남자', '7번방의 선물', '국제시장' 등이 있다. 또한 연극, TV 프로그램, 영화 등 다양한 매체에서 활약했으며, 여러 상을 수상했다.<|eot_id|>


## Inference before Fine-Tuning

In [30]:
def prompt_template(system, user):
    return (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        f"{system}<|eot_id|>"
        
        "<|start_header_id|>user<|end_header_id|>\n\n"
        f"{user}<|eot_id|>"
        
        "<|start_header_id|>assistant<|end_header_id|>\n\n"
    )

In [31]:
def generate_response(system ,user):
    prompt = prompt_template(system, user)
    
    input_ids = korean_tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=korean_tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return korean_tokenizer.decode(outputs[0], skip_special_tokens=False)

In [32]:
#system_prompt = "You are a helpful assistant. Respond to the following user prompt."
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."

In [33]:
#user_prompt = "Write me a poem about Machine Learning."
user_prompt = "머신러닝에 대한 시를 써주세요."

In [34]:
response = generate_response(system_prompt, user_prompt)
print(response)

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음 지시사항에 대한 응답을 작성해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

머신러닝에 대한 시를 써주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's a poem about jogging:

The sun rises high and bright,
I lace up my shoes, and take flight.
The pavement stretches far and wide,
A path that beckons me to glide.

With every step, I feel alive,
My heart beats fast, my spirit thrives.
The rhythm of my footsteps sweet,
Echoes through the morning street.

The world awakens, fresh and new,
As I jog on, with nothing to pursue.
No worries weigh upon my mind,
Just the rush of endorphins' design.

The wind whispers secrets in my ear,
Of freedom found, and joy so clear.
The trees sway gently, their leaves rustle too,
As I run on, with a heart anew.

With each mile marker, I feel strong,
My body moving, my soul taking song.
The world is mine, as I hit the ground,
Free and wild, without a sound.

So here I'll stay, in this jogging haz

## Supervised Fine-Tuning (LoRA)

In [35]:
lora_config = LoraConfig(
    task_type=task_type,
    target_modules=target_modules,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias=bias
)

In [36]:
# Number of trainable parameters
model = get_peft_model(model, lora_config)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters (in billions): {num_params / 1e9:.2f}")
print(f"Percentage of trainable parameters: {num_params / model.num_parameters() * 100:.2f}%")

Number of trainable parameters (in billions): 0.04
Percentage of trainable parameters: 0.51%


In [37]:
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    save_strategy=save_strategy,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    save_total_limit=save_total_limit,
    report_to=report_to,
    
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    optim=optim,
    weight_decay=weight_decay
)

In [38]:
trainer = SFTTrainer(
    model=model,
    tokenizer=korean_tokenizer,
    peft_config=lora_config,
    args=training_args,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=packing
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
wandb.finish()
trainer.save_model(model_name)

## Inference after Fine-Tuning

In [ ]:
#system_prompt = "You are a helpful assistant. Respond to the following user prompt."
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."

In [ ]:
#user_prompt = "Write me a poem about Machine Learning."
user_prompt = "머신러닝에 대한 시를 써주세요."

In [ ]:
response = generate_response(system_prompt, user_prompt)
print(response)

## Upload Model

In [ ]:
# Flush memory
import gc
gc.collect()

del trainer, model
torch.cuda.empty_cache()

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(korean_tokenizer_id)

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
model = PeftModel.from_pretrained(base_model, model_name)
model = model.merge_and_unload()

In [ ]:
# Push model and tokenizer to Hugging Face Hub
tokenizer.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)
model.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)